In [1]:
#! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
#! pip install optuna

In [2]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://reddit-reccomender-bucket/11', creation_time=1763396487435, experiment_id='11', last_update_time=1763396487435, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm  import SVC
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

## Step 2: Train-test split BEFORE sampling
#Step 0:train test split
X_train_raw,X_test_raw,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
)

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000

vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply Random Undersampler to train only
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, y_train)



#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparision")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test_vec)

    return accuracy_score(y_test,preds)


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train_res, X_test_vec, y_train_res, y_test)

# Run the experiment for SVM
run_optuna_experiment()




[I 2025-11-18 09:08:48,705] A new study created in memory with name: no-name-f0b10fc8-3827-4758-bb86-f24f73c52225
[I 2025-11-18 09:09:23,617] Trial 0 finished with value: 0.604527478521751 and parameters: {'C': 0.004169077937522181, 'kernel': 'linear'}. Best is trial 0 with value: 0.604527478521751.
[I 2025-11-18 09:10:09,802] Trial 1 finished with value: 0.46270285012955137 and parameters: {'C': 0.00011929225931509751, 'kernel': 'rbf'}. Best is trial 0 with value: 0.604527478521751.
[I 2025-11-18 09:10:45,943] Trial 2 finished with value: 0.6963043774717033 and parameters: {'C': 0.25176940783752694, 'kernel': 'rbf'}. Best is trial 2 with value: 0.6963043774717033.
[I 2025-11-18 09:11:20,874] Trial 3 finished with value: 0.6238919950906859 and parameters: {'C': 0.005251947405542825, 'kernel': 'linear'}. Best is trial 2 with value: 0.6963043774717033.
[I 2025-11-18 09:12:01,744] Trial 4 finished with value: 0.7804445656620755 and parameters: {'C': 0.35526777925557895, 'kernel': 'linear'

🏃 View run SVM_Undersampling_BOW_Trigram at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11/runs/30e027c110f6405c8b3862c4ad3d0daf
🧪 View experiment at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11
